In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

## Imports

In [ ]:
import os
import re
import cv2
import time
import json
import torch
import imageio
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from collections import Counter
from tqdm.notebook import tqdm
from skimage.transform import resize

In [ ]:
from params import *

In [ ]:
from data.dataset import NFLDatasetDet
from data.transforms import get_transfos_det

from utils.plot import plot_bboxes
from utils.save import load_pickle
from utils.torch import load_effdet_weights

from model_zoo.models_det import get_inference_model

## Data

In [ ]:
df_train = pd.read_csv(DATA_PATH + 'df_train.csv')

frame_impacts = df_train[["image_name", "extended_impact"]].groupby('image_name').max().reset_index()
frame_impacts = frame_impacts.rename(columns={"extended_impact": "frame_has_impact"})
df_train = df_train.merge(frame_impacts, on="image_name")

# df_train = df_train[df_train['frame_has_impact'] == 1]
df_train['extended_impact'] += 1 
df_train['impact'] +=1 

folds = pd.read_csv(OUT_DIR + "folds.csv")
df_train = df_train.merge(folds, on="video")

In [ ]:
if "512" in IMG_PATH:
    df_train["x"] = (df_train["x"] / IMG_SHAPE[1] * SIZE).astype(int)
    df_train["y"] = (df_train["y"] / IMG_SHAPE[0] * SIZE).astype(int)
    df_train["w"] = (df_train["w"] / IMG_SHAPE[1] * SIZE).astype(int)
    df_train["h"] = (df_train["h"] / IMG_SHAPE[0] * SIZE).astype(int)

## Model

In [ ]:
# CP_FOLDER = LOG_PATH + "2020-12-07/15/"  # d3, best
# CP_FOLDER = LOG_PATH + "2020-12-08/3/"  # d4
# CP_FOLDER = LOG_PATH + "2020-12-09/10/"  # d4
CP_FOLDER = LOG_PATH + "2020-12-14/1/"

In [ ]:
os.listdir(CP_FOLDER)

In [ ]:
FOLD = 0

In [ ]:
meter_ref = load_pickle(CP_FOLDER + f'meter_{FOLD}.pkl')

In [ ]:
config = json.load(open(CP_FOLDER + f'config.json', 'r'))

In [ ]:
config

### Model

In [ ]:
model = get_inference_model(config['name'], config['num_classes'])

In [ ]:
model = load_effdet_weights(model, CP_FOLDER + f'{config["name"]}_{FOLD}.pt')

### Dataset

In [ ]:
df_val = df_train.iloc[meter_ref.val_idx].reset_index(drop=True)
# games = df_val['gameKey'].unique()
# df_val = df[df['gameKey'].apply(lambda x: x in games)]

In [ ]:
dataset = NFLDatasetDet(
    df_val,
    transforms=get_transfos_det(train=False),
    root=IMG_PATH,
    train=False
)

## Inference

In [ ]:
from training.train import collate_fn
from training.meter import NFLMeter

In [ ]:
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader

In [ ]:
def predict(model, dataset, batch_size, device="cuda", num_classes=1):
    model = model.eval().to(device)
    
    loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=collate_fn,
        num_workers=NUM_WORKERS,
        pin_memory=True,
    )
    
    meter = NFLMeter(num_classes=num_classes)
    
    with torch.no_grad():
        for batch in tqdm(loader):
            images = torch.stack(batch[0]).to(device)
            scales = torch.tensor([1] * len(images)).long().to(device)

            y_pred = model(images, scales).detach()

            meter.update(batch, y_pred) 

    return meter

In [ ]:
meter = predict(
    model,
    dataset,
    config['val_bs'],
    num_classes=config['num_classes']
)

In [ ]:
preds_helmet, truths_helmet, preds_impact, t = meter.get_helmets(helmet_threshold=0.25)

In [ ]:
meter.compute_scores()

### Impact detection ratio

In [ ]:
preds_helmet, _, _, truths_impact = meter.get_helmets(helmet_threshold=0.1, impact_threshold=0.)

In [ ]:
from utils.metrics import detection_ratio

In [ ]:
detection_ratio(preds_helmet, truths_impact)

In [ ]:
meter.num_classes = 1

## ROI

In [ ]:
df_train = pd.read_csv(DATA_PATH + 'df_train.csv')

frame_impacts = df_train[["image_name", "extended_impact"]].groupby('image_name').max().reset_index()
frame_impacts = frame_impacts.rename(columns={"extended_impact": "frame_has_impact"})
df_train = df_train.merge(frame_impacts, on="image_name")

# df_train = df_train[df_train['frame_has_impact'] == 1]
df_train['extended_impact'] += 1 
df_train['impact'] +=1 

folds = pd.read_csv(OUT_DIR + "folds.csv")
df_train = df_train.merge(folds, on="video")

In [ ]:
df_val = df_train.iloc[meter_ref.val_idx].reset_index(drop=True)

In [ ]:
rois = np.load(CP_FOLDER + "rois.npy")

In [ ]:
t = get_transfos_det(train=False, visualize=False)

In [ ]:
t.

In [ ]:
from torch.utils.data import Dataset
import albumentations as albu
from albumentations.pytorch.transforms import ToTensorV2


class NFLDatasetDetROI(Dataset):
    def __init__(self, df, rois, root="", transforms=None, train=False):
        super().__init__()
        self.train = train
        self.root = root
        self.rois = rois

        self.transforms = transforms
        self.bbox_params = albu.BboxParams(
            format="pascal_voc", min_area=0, min_visibility=0, label_fields=["labels"]
        )
        
        self.images = df.image_name.unique()
        df = df.copy()
        df['w'] += df['x']
        df['h'] += df['y']
        group = (
            df[["image_name", "x", "y", "w", "h", "impact", "extended_impact"]]
            .groupby("image_name")
            .agg(list)
            .reset_index()
        )
        self.images = group["image_name"].values

        if self.train:
            self.labels = group['extended_impact'].values.tolist()
        else:
            self.labels = group['impact'].values.tolist()

        self.boxes = group[['x', 'y', 'w', 'h']].values
        self.boxes = [np.array(box.tolist()).T for box in self.boxes]
        
    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = cv2.imread(f"{self.root}/{self.images[idx]}", cv2.IMREAD_COLOR)
        boxes = self.boxes[idx]
        labels = self.labels[idx]
        roi = self.rois[idx]

        frame = int(self.images[idx][-8:-4])
        vid = self.images[idx][:-9]

        transforms = albu.Compose([
            albu.Crop(roi[0], roi[1], roi[2], roi[3]), 
            self.transforms,
        ], bbox_params=self.bbox_params,)

        sample = transforms(
            **{"image": image, "bboxes": boxes, "labels": labels}
        )
        
        image = sample["image"]
        labels = sample["labels"]

        boxes = torch.stack(
            tuple(map(torch.tensor, zip(*sample["bboxes"])))
        ).permute(1, 0)

        boxes[:, [0, 1, 2, 3]] = boxes[:, [1, 0, 3, 2]]  # (x, y), -> (y, x)

        return image, boxes.float(), torch.tensor(labels), vid, frame, roi

In [ ]:
dataset_roi_viz = NFLDatasetDetROI(
    df_val.copy(), 
    rois, 
    transforms=get_transfos_det(train=False, visualize=True),
    root=IMG_PATH_F
)

In [ ]:
# for i in np.random.choice(len(dataset_roi_viz), 10):
#     image, boxes, label, vid, frame, roi = dataset_roi_viz[i]
#     boxes = boxes.int()

#     plt.figure(figsize=(8, 8))
#     plot_bboxes(image.numpy().transpose(1, 2, 0).copy(), boxes, label, transpose=True)
#     plt.show()

In [ ]:
dataset_roi = NFLDatasetDetROI(
    df_val.copy(), 
    rois, 
    transforms=get_transfos_det(train=False, visualize=False),
    root=IMG_PATH_F
)

In [ ]:
meter_roi = predict(
    model,
    dataset_roi,
    config['val_bs'],
    num_classes=config['num_classes']
)

In [ ]:
meter_roi.compute_scores()

In [ ]:
from utils.save import load_pickle
from utils.metrics import iou_score

In [ ]:
vids = list(np.unique(df_val['video']))
preds_helmet, truths_helmet, preds_impact, _ = meter_roi.get_helmets(helmet_threshold=0.25)

In [ ]:
TOT = 0
DET = 0
PLOT = True

# for idx in np.random.choice(len(dataset), 1000):
for idx in tqdm(range(len(dataset_roi_viz))):
    image, boxes, label, vid, frame, roi = dataset_roi_viz[idx]
    
#     if "Sideline" in vid:
#         continue
        
    boxes = boxes.int()

    boxes_pred = preds_helmet[vids.index(vid + ".mp4")]
    boxes_pred = boxes_pred[:, 1:][boxes_pred[:, 0] == frame]
    label_pred = [1] * len(boxes_pred)
    boxes_pred = boxes_pred.astype(int)
    
    total, detected = 0, 0
    for b in boxes[label == 2]:
        total += 1
        box = b.numpy()[np.array([1, 0, 3, 2])]
        for i in range(len(boxes_pred)):
            iou = iou_score(box, boxes_pred[i])
            if iou > 0.35:
                label_pred[i] = 2
                detected += 1
                break

    TOT += total
    DET += detected
    
    if PLOT and total != detected:
        if label.max() == 2:
            plt.figure(figsize=(15, 7))
            plt.subplot(1, 2, 1)
            plot_bboxes(image.numpy().transpose(1, 2, 0).copy(), boxes, label, transpose=True)
            plt.title('Truth')
            plt.axis(False)

            plt.subplot(1, 2, 2)
            plot_bboxes(image.numpy().transpose(1, 2, 0).copy(), boxes_pred, label_pred, transpose=False)
            plt.title('Pred')
            plt.axis(False)
            plt.show()

            print(f'Detected {detected} / {total} helmets with impacts')
    
#             break

In [ ]:
TOT, DET

In [ ]:
257 / 415